In [1]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint

In [2]:
# URL
renew_energy_url = "https://api.worldbank.org/v2/en/country/all/indicator/EG.FEC.RNEW.ZS?format=json&per_page=20000&source=2"
cntry_url = 'https://restcountries.eu/rest/v2/all'

In [31]:
# Get Renewable Energy Consumption Data
renew_energy_response = requests.get(renew_energy_url).json()
reo = renew_energy_response[1]

# building percent of renewable energy dataframe 
#starting with country
country = [reo[i]['country']['value'] for i in range(len(reo))]
renew_energy_df = pd.DataFrame({'Country':country})

# adding year column to dataframe
year= [reo[i]['date'] for i in range(len(reo))]
renew_energy_df["Year"]= year

# adding alphacode column to dataframe
a_code = [reo[i]['countryiso3code'] for i in range(len(reo))]
renew_energy_df["alpha3code"]= a_code

# adding Percent Renewable value to dataframe
percent_renewable = [reo[i]['value'] for i in range(len(reo))]
renew_energy_df["Percent Renewable"]= percent_renewable

# dropping rows with blank alphacodes - maybe this will do the trick to remove all "non-countries"
renew_energy_df = renew_energy_df[renew_energy_df.alpha3code != ""]

# reomving all nan values for percent renewable
renew_energy_df = renew_energy_df[np.isfinite(renew_energy_df['Percent Renewable'])]

# Changing Year column dtype to float

renew_energy_df["Year"] = pd.to_numeric(renew_energy_df["Year"])

# limiting years to 2000-2015

renew_energy_df = renew_energy_df.drop(renew_energy_df[(renew_energy_df.Year < 2000) | (renew_energy_df.Year > 2015)].index)
renew_energy_df = renew_energy_df.drop(columns=["alpha3code"])
renew_energy_df.head()

,Country,Year,Percent Renewable
2776,Afghanistan,2015,18.423477
2777,Afghanistan,2014,19.314269
2778,Afghanistan,2013,16.334293
2779,Afghanistan,2012,13.973586
2780,Afghanistan,2011,11.482706


In [37]:
# Get Countries, Lat, Long Data
cntry_response = requests.get(cntry_url).json()

# Process each country and store details in a list
cntry_ls = []
for index, each in enumerate(cntry_response):
    try:
        c_name = each['name']
        c_code = each['alpha3Code']
        c_lat = each['latlng'][0]
        c_lng = each['latlng'][1]
        c_pop = each['population']
        c_area = each['area']
        c_details = [c_name, c_code, c_pop, c_area, c_lat, c_lng]
        cntry_ls.append(c_details)
    except:
        print(f"Data not found for {c_name}")

# Convert list to dataframe        
cntry_df = pd.DataFrame(cntry_ls, columns=["Country", "Code", "Population", "Area", "Lat", "Lng"])

# Convert data to numeric
cntry_df["Population"] = pd.to_numeric(cntry_df["Population"])
cntry_df["Area"] = pd.to_numeric(cntry_df["Area"])
cntry_df["Lat"] = pd.to_numeric(cntry_df["Lat"])
cntry_df["Lng"] = pd.to_numeric(cntry_df["Lng"])

# Cross Join year
years = []
for i in range(2000, 2016, 1):
    years.append(i)
years_df = pd.DataFrame(years, columns=["Year"])
years_df

cntry_df['dummykey'] = 0
years_df['dummykey'] = 0

countries_df = cntry_df.merge(years_df, how='outer')

countries_df = countries_df.drop(columns=["dummykey"])
countries_df.head()

Data not found for United States Minor Outlying Islands


,Country,Code,Population,Area,Lat,Lng,Year
0,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2000
1,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2001
2,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2002
3,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2003
4,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2004


In [40]:
# Get Emission data
emm_df = pd.read_csv("EmissionData.csv", skiprows=4)

# Remove unwanted columns
emm_df = emm_df.drop(columns=["Indicator Name", "Indicator Code"])
emm_df = emm_df.drop(columns=["1960", "1961", "1962", "1963", "1964", "1965", "1966", "1967", "1968", "1969"])
emm_df = emm_df.drop(columns=["1970", "1971", "1972", "1973", "1974", "1975", "1976", "1977", "1978", "1979"])
emm_df = emm_df.drop(columns=["1980", "1981", "1982", "1983", "1984", "1985", "1986", "1987", "1988", "1989"])
emm_df = emm_df.drop(columns=["1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999"])
emm_df = emm_df.drop(columns=["2016", "2017", "2018", "Unnamed: 63"])


# Unpivot: Change year columns to rows
emm_df = pd.melt(emm_df, id_vars=['Country Name'], value_vars=['2000', '2001', '2002', '2003', '2004', '2005',
                                                       '2006', '2007', '2008', '2009', '2010', '2011',
                                                       '2012', '2013', '2014', '2015'])
emm_df.columns=["Country", "Year", "Emission"]
emm_df = emm_df.dropna()
emm_df["Year"] = pd.to_numeric(emm_df["Year"])

In [34]:
# Get Education Data
Ed_df = pd.read_csv("college_ed.csv")

Ed_df.groupby(['Country']).head()

Ed_df = Ed_df.drop(Ed_df[(Ed_df.Year < 2000) | (Ed_df.Year > 2015)].index)
Ed_df = Ed_df.drop(columns=["Code"])
Ed_df.head()

,Country,Year,College Enrolment ratio
14,Afghanistan,2003,1.25684
15,Afghanistan,2004,1.25603
16,Afghanistan,2009,3.90339
17,Afghanistan,2011,3.75598
18,Afghanistan,2014,8.66280


In [49]:
# Get GDP Data
gdp_file = 'gdp.csv'
gdp_df = pd.read_csv(gdp_file)

gdp_df.drop(['1960', '1961','1962','1963','1964',
         '1965', '1966','1967', '1968','1969',
         '1970','1971','1972', '1973','1974',
         '1975','1976','1977','1978','1979',
         '1980','1981','1982','1983','1984',
         '1985','1986', '1987','1988','1989',
         '1990', '1991','1992','1993','1994',
         '1995', '1996','1997','1998','1999',
         '2016','2017','2018'], axis=1, inplace=True)

del gdp_df['Indicator Code']
del gdp_df['Indicator Name']

gdp_df =pd.melt(gdp_df, id_vars=['Country Name','Country Code'],value_vars=['2000', '2001', '2002', '2003', '2004', '2005',
                                                                       '2006', '2007', '2008', '2009', '2010', '2011',
                                                                       '2012', '2013', '2014', '2015'])

gdp_df.drop( gdp_df[gdp_df ['value'] == 0.000000 ].index , inplace=True)

gdp_df = gdp_df.rename({'Country Name': 'Country'}, axis=1)
gdp_df = gdp_df.rename({'value': 'GDP'}, axis=1)
gdp_df = gdp_df.rename({'variable': 'Year'}, axis=1)
gdp_df = gdp_df.dropna(how='any')
gdp_df = gdp_df.drop(columns=["Country Code"])
gdp_df["Year"] = pd.to_numeric(gdp_df["Year"])

In [77]:
# Create results with all data combined
result1 = countries_df.merge(renew_energy_df, how='left', on=['Country','Year'])
result2 = result1.merge(emm_df, how='left', on=['Country','Year'])
result3 = result2.merge(Ed_df, how='left', on=['Country','Year'])
result = result3.merge(gdp_df, how='left', on=['Country','Year'])
result['GDP'] = result['GDP']/100000

# Drop rows if Percent Renewable is NA
result = result.dropna(subset=['Percent Renewable'])
result

,Country,Code,Population,Area,Lat,Lng,Year,Percent Renewable,Emission,College Enrolment ratio,GDP
0,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2000,54.243126,0.038506,NaN,NaN
1,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2001,54.055055,0.039002,NaN,NaN
2,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2002,43.771149,0.048716,NaN,40551.795663
3,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2003,42.276141,0.051830,1.256840,45155.588081
4,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2004,49.843148,0.039378,1.256030,52267.788089
5,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2005,40.859171,0.052948,NaN,62091.376248
6,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2006,37.137249,0.063728,NaN,69712.855947
7,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2007,33.862579,0.085418,NaN,97478.795319
8,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2008,21.343708,0.154101,NaN,101092.258137
9,Afghanistan,AFG,27657145,652230.0,33.0,65.0,2009,17.813855,0.241723,3.903390,124390.870768
